In [1]:
# 制作数据集
import sys
import os
root = "/data/public/renhaoye/morphics/"
subdirs = [os.path.join(root, d) for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
for subdir in subdirs:
    sys.path.append(subdir)

In [11]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import random
from torch.backends import cudnn
from dataset.galaxy_dataset import *
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Subset
from utils import schemas
from training import losses, metrics
import argparse
from utils.utils import *
from torchvision.models import *
from bayesian_torch.models.dnn_to_bnn import dnn_to_bnn, get_kl_loss
from models.morphics import Morphics
from torch.optim.lr_scheduler import ReduceLROnPlateau
from models.utils import *
img = torch.Tensor(load_img("/data/public/renhaoye/morphics/dataset/in_decals/agmtn/119.85429851358965_28.803799075693085.fits"))
# img = img.unsqueeze(0)
img.shape

torch.Size([3, 256, 256])

In [12]:
model = torch.load("/data/public/renhaoye/morphics/pth1/model_1.pt")

In [14]:
predict, stn = model(img.unsqueeze(0).cuda())
fits.writeto(f'/data/public/renhaoye//1.fits', stn[0].cpu().detach().numpy(), overwrite=True)

In [7]:
question_answer_pairs = gz2_pairs
dependencies = gz2_and_decals_dependencies
schema = schemas.Schema(question_answer_pairs, dependencies)
predict = softmax_output(predict,schema.question_index_groups)

In [15]:
predict

tensor([[ 0.4397,  0.4485, -0.3532, -1.1289,  1.1948, -0.2392,  0.0175, -0.7089,
          0.1281,  1.3058, -0.8664, -0.5173,  1.2402,  1.1093, -0.7388, -0.2065,
          0.0655, -0.8323, -0.5023,  0.2214,  1.5271,  0.6526,  0.5741, -0.7883,
         -0.6598,  1.7232, -0.9318, -1.3321, -0.3257,  0.3750,  2.5532,  0.9620,
         -1.6030, -0.9275]], device='cuda:0', grad_fn=<AddBackward0>)

In [17]:
pred_mnist, epi_mnist_norm, ale_mnist_norm = get_uncertainty_per_image(model, img.cuda(), T=10, normalized=True)

In [18]:
pred_mnist

array([ 6.1986589e-01,  2.0184496e-01, -1.4715673e+00, -7.7971363e-01,
        6.9392610e-01,  2.2599626e-01, -2.0834167e-01, -9.6987319e-01,
       -1.1796487e-01,  8.3403474e-01, -2.5028355e+00, -5.7288098e-01,
        9.3109423e-01,  7.5418860e-01, -1.1366481e+00,  1.2420118e-04,
        5.2260745e-01, -8.3282900e-01, -1.1405461e+00, -3.4915301e-01,
        1.0287037e+00,  4.4249696e-01,  1.6260603e-01, -3.6677971e-01,
       -1.1561434e+00,  1.3320980e+00, -4.6493351e-01, -1.6384544e+00,
       -2.6482787e+00,  4.7209993e-01,  1.4303304e+00, -4.2549528e-02,
       -1.3607514e+00, -1.2942281e+00], dtype=float32)

In [19]:
question_answer_pairs = gz2_pairs
dependencies = gz2_and_decals_dependencies
schema = schemas.Schema(question_answer_pairs, dependencies)
pred_mnist = softmax_output(torch.Tensor(pred_mnist,schema.question_index_groups)
pred_mnist

tensor([ 60.6979,  40.3021,   0.0000, 100.0000,   0.0000, 100.0000,   0.0000,
         30.6042,  70.3958,   0.0000,   2.5264,  11.5155,  48.3149,  40.6432,
          0.0000,  37.8549,  63.1451,   0.0000,  31.8751,  69.1249,   0.0000,
         57.3824,  43.6176,   0.0000,   7.2349,  76.0683,  13.4456,   4.8491,
          2.4022,   0.0000,  77.7074,  18.5863,   5.7064,   0.0000])

array([1.7033335e-08, 1.6753219e-07, 5.5029257e-09, 1.5647780e-07,
       3.2251464e-08, 1.8313497e-07, 1.7248030e-07, 2.9690239e-07,
       2.4569706e-06, 1.7428664e-07, 2.0172670e-06, 5.1425877e-13,
       2.9239136e-08, 5.5045973e-08, 1.0053172e-07, 2.4659010e-07,
       2.0776032e-08, 1.0866687e-08, 3.7976289e-08, 3.5593781e-09,
       1.1430995e-08, 1.6909146e-06, 2.3880619e-07, 1.8421627e-08,
       9.2632142e-08, 1.3757867e-07, 8.0269942e-08, 3.5363635e-06,
       8.3060506e-08, 9.3242562e-08, 1.9771420e-07, 4.2517727e-09,
       4.4667114e-08, 1.7839776e-08], dtype=float32)